In [2]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from itertools import count
from collections import defaultdict
from scipy.sparse import csr
import numpy as np

def vectorize_dic(dic, ix=None, p=None):
    if(ix == None):
        d = count(0)
        ix = defaultdict(lambda:next(d))
        
    n = len(list(dic.values())[0]);print('n ',n)
    g = len(list(dic.keys()));print('g ',g)
    nz = n*g;print(nz)
    
    col_ix = np.empty(nz, dtype=int);print('col_ix.shape ', col_ix.shape)
    
    i = 0
    for k, lis in dic.items():
        col_ix[i::g] = [ix[str(k)+str(el)] for el in lis]
        i += 1
    
    row_ix = np.repeat(np.arange(n), g);print('row_ix.shape ', row_ix.shape)
    data = np.ones(nz);print('data.shape ', data.shape)
    
    print('col_ix', col_ix)
    print('row_ix', row_ix)
    print('ix', ix)
    print('len(ix)', len(ix))
    
    if(p == None):
        p = len(ix)
        
    print('p', p)
    ixx = np.where(col_ix < p);print('ixx ', ixx)
    
    return csr.csr_matrix(
        (data[ixx], (row_ix[ixx], col_ix[ixx])), shape=(n, p)), ix

In [3]:
dic = {'age':[3,2,3,4],'sex':['boy','girl','boy','boy'],'name':['a','b','c','d']}
ans = vectorize_dic(dic)


n  4
g  3
12
col_ix.shape  (12,)
row_ix.shape  (12,)
data.shape  (12,)
col_ix [0 3 5 1 4 6 0 3 7 2 3 8]
row_ix [0 0 0 1 1 1 2 2 2 3 3 3]
ix defaultdict(<function vectorize_dic.<locals>.<lambda> at 0x7f54c01b5268>, {'age3': 0, 'age2': 1, 'age4': 2, 'sexboy': 3, 'sexgirl': 4, 'namea': 5, 'nameb': 6, 'namec': 7, 'named': 8})
len(ix) 9
p 9
ixx  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),)


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

# load train and test data
cols = ['user','item','rating','timestamp']
train = pd.read_csv('../../../Datasets/MovieLens/ml-100k/ua.base', delimiter='\t', names=cols)
test = pd.read_csv('../../../Datasets/MovieLens/ml-100k/ua.test', delimiter='\t', names=cols)

x_train, ix = vectorize_dic({'users':train.user.values,'items':train.item.values})
x_test, ix = vectorize_dic({'users':test.user.values, 'items':test.item.values}, ix, x_train.shape[1])

y_train = train.rating.values
y_test = test.rating.values

n  90570
g  2
181140
col_ix.shape  (181140,)
row_ix.shape  (181140,)
data.shape  (181140,)
col_ix [   0  943    0 ... 2154  942 2265]
row_ix [    0     0     1 ... 90568 90569 90569]
ix defaultdict(<function vectorize_dic.<locals>.<lambda> at 0x7f54d03ecbf8>, {'users1': 0, 'users2': 1, 'users3': 2, 'users4': 3, 'users5': 4, 'users6': 5, 'users7': 6, 'users8': 7, 'users9': 8, 'users10': 9, 'users11': 10, 'users12': 11, 'users13': 12, 'users14': 13, 'users15': 14, 'users16': 15, 'users17': 16, 'users18': 17, 'users19': 18, 'users20': 19, 'users21': 20, 'users22': 21, 'users23': 22, 'users24': 23, 'users25': 24, 'users26': 25, 'users27': 26, 'users28': 27, 'users29': 28, 'users30': 29, 'users31': 30, 'users32': 31, 'users33': 32, 'users34': 33, 'users35': 34, 'users36': 35, 'users37': 36, 'users38': 37, 'users39': 38, 'users40': 39, 'users41': 40, 'users42': 41, 'users43': 42, 'users44': 43, 'users45': 44, 'users46': 45, 'users47': 46, 'users48': 47, 'users49': 48, 'users50': 49, 'users51

ixx  (array([    0,     1,     2, ..., 18857, 18858, 18859]),)


In [5]:
print(x_train[:5])

x_train = x_train.todense()
x_test = x_test.todense()

print(x_train.shape)
print(x_test.shape)

  (0, 0)	1.0
  (0, 943)	1.0
  (1, 0)	1.0
  (1, 944)	1.0
  (2, 0)	1.0
  (2, 945)	1.0
  (3, 0)	1.0
  (3, 946)	1.0
  (4, 0)	1.0
  (4, 947)	1.0
(90570, 2623)
(9430, 2623)


In [9]:
print(x_train[:5],x_test[:5])

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]] [[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [11]:
import tensorflow as tf

n, p = x_train.shape
# number of latent factors
k = 10
# design features of users
X = tf.placeholder('float', shape=[None, p])
# target vector
Y = tf.placeholder('float', shape=[None, 1])

# bias and weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))

# matrix factorization factors, randomly initialized
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))

# estimation of y, initialized to 0
Y_hat = tf.Variable(tf.zeros([n, 1]))

/home/wyl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
# calculate output with FM equation
linear_terms = tf.add(w0, tf.reduce_sum(tf.multiply(W, X), 1, keep_dims=True))
print(W.shape, X.shape)
pair_interactions = tf.multiply(0.5, 
                                tf.reduce_sum(
                                    tf.subtract(
                                        tf.pow(tf.matmul(X,tf.transpose(V)), 2), 
                                        tf.matmul(tf.pow(X, 2), tf.pow(tf.transpose(V), 2))), 
                                    1, keepdims=True))
Y_hat = tf.add(linear_terms, pair_interactions)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
(2623,) (?, 2623)


In [13]:
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

print('W',W.shape, 'V', V.shape)
l2_norm = tf.reduce_sum(tf.multiply(lambda_w, tf.pow(W,2)))+\
tf.reduce_sum(tf.multiply(lambda_v, tf.pow(V,2)))
print('l2_norm', l2_norm.shape)
error = tf.reduce_mean(tf.square(tf.subtract(Y, Y_hat)))
print('Y', Y.shape, 'Y_hat', Y_hat.shape, 'error', error.shape)
loss =  tf.add(error, l2_norm)
print('loss', loss.shape)

W (2623,) V (10, 2623)
l2_norm ()
Y (?, 1) Y_hat (?, 1) error ()
loss ()


In [14]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

In [15]:
def batcher(X, Y=None, batch_size=-1):
    n_samples = X.shape[0]
    
    if(batch_size == -1):
        batch_size = n_samples
    if(batch_size < 1):
        raise ValueError('Parameter batch_size={} is unsupported'.format(batch_size))
    for i in range(0, n_samples, batch_size):
        upper_bound = min(i+batch_size, n_samples)
        ret_x = X[i:upper_bound]
        ret_y = Y[i:upper_bound]
        yield(ret_x, ret_y)

In [16]:
from tqdm import tqdm_notebook as tqdm

epoches = 2
batch_size = 100

init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

for epoch in tqdm(range(epoches), unit='epoch'):
    perm = np.random.permutation(x_train.shape[0])
    for bx, by in batcher(x_train[perm], y_train[perm], batch_size):
        sess.run(optimizer, feed_dict={
            X:bx.reshape(-1, p),
            Y:by.reshape(-1, 1)
        })

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

In [17]:
errors = []
for bx, by in batcher(x_test, y_test):
    errors.append(sess.run(error, feed_dict={
        X:bx.reshape(-1, p),
        Y:by.reshape(-1, 1)
    }))
RMSE = np.sqrt(np.array(errors).mean())
print(RMSE)

1.1060351


In [18]:
sess.close()